In [ ]:
import os
import zipfile
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd

# ========= 1. 解壓縮資料與模型 =========
data_zip = '/content/processed_data.zip'
lstm_zip = '/content/models_lstm.zip'
timesnet_zip = '/content/models_timesnet.zip'

data_path = '/content/data/processed/'
model_path = '/content/models/'
os.makedirs(data_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

with zipfile.ZipFile(data_zip, 'r') as zf:
    zf.extractall(data_path)
with zipfile.ZipFile(lstm_zip, 'r') as zf:
    zf.extractall('/')
with zipfile.ZipFile(timesnet_zip, 'r') as zf:
    zf.extractall('/')

print("資料與模型已解壓完成")

# ========= 2. 載入資料 =========
X = np.load(f"{data_path}/X_train.npy")
Y = np.load(f"{data_path}/Y_train.npy")

mask = ~np.isnan(X).any(axis=(1, 2)) & ~np.isnan(Y).any(axis=1)
X = X[mask]
Y = Y[mask]

X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ========= 3. 定義模型 =========
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

class SimpleTimesNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleTimesNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.linear2(self.relu(self.linear1(x)))

# ========= 4. 載入模型 =========
input_dim = X.shape[2]
seq_len = X.shape[1]
output_dim = Y.shape[1]

lstm_model = LSTMModel(input_dim, hidden_dim=64, output_dim=output_dim).to(device)
lstm_model.load_state_dict(torch.load("/content/models/lstm_model.pth", map_location=device))
lstm_model.eval()

timesnet_model = SimpleTimesNet(seq_len * input_dim, 128, output_dim).to(device)
timesnet_model.load_state_dict(torch.load("/content/models/timesnet_model.pth", map_location=device))
timesnet_model.eval()

print("模型載入完成")

# ========= 5. 預測 =========
with torch.no_grad():
    X_input = X_tensor.to(device)
    lstm_pred = lstm_model(X_input).cpu().numpy()
    timesnet_pred = timesnet_model(X_input).cpu().numpy()
    ground_truth = Y_tensor.cpu().numpy()

# ========= 6. 評估指標 =========
def evaluate(true, pred):
    mae = mean_absolute_error(true, pred)
    rmse = np.sqrt(mean_squared_error(true, pred))
    r2 = r2_score(true, pred)
    return mae, rmse, r2

lstm_mae, lstm_rmse, lstm_r2 = evaluate(ground_truth, lstm_pred)
timesnet_mae, timesnet_rmse, timesnet_r2 = evaluate(ground_truth, timesnet_pred)

print(f"\nLSTM -> MAE: {lstm_mae:.4f}, RMSE: {lstm_rmse:.4f}, R²: {lstm_r2:.4f}")
print(f"TimesNet -> MAE: {timesnet_mae:.4f}, RMSE: {timesnet_rmse:.4f}, R²: {timesnet_r2:.4f}")

# ========= 7. 圖表：預測 vs 實際 =========
rainy_idx = np.where(ground_truth[:, 0] > 0)[0]
start_idx = rainy_idx[0] if len(rainy_idx) > 0 else 0
end_idx = min(start_idx + 200, len(ground_truth))

plt.figure(figsize=(14, 6))
plt.plot(ground_truth[start_idx:end_idx, 0], label='Actual')
plt.plot(timesnet_pred[start_idx:end_idx, 0], '--', label='TimesNet')
plt.plot(lstm_pred[start_idx:end_idx, 0], ':', label='LSTM')
plt.title('Rainfall Prediction (First Rainy Segment)')
plt.xlabel('Time Step')
plt.ylabel('Precp')
plt.legend()
plt.grid()
plt.savefig('/content/rainy_segment.png')
plt.show()

# ========= 8. 誤差分佈圖 =========
lstm_error = ground_truth[:, 0] - lstm_pred[:, 0]
timesnet_error = ground_truth[:, 0] - timesnet_pred[:, 0]

plt.figure(figsize=(10, 5))
plt.hist(lstm_error, bins=50, alpha=0.5, label='LSTM Error')
plt.hist(timesnet_error, bins=50, alpha=0.5, label='TimesNet Error')
plt.title('Prediction Error Distribution')
plt.xlabel('Error (Actual - Predicted)')
plt.ylabel('Frequency')
plt.legend()
plt.grid()
plt.savefig('/content/error_distribution.png')
plt.show()

# ========= 9. 儲存成績表格 =========
results = pd.DataFrame({
    'Model': ['LSTM', 'TimesNet'],
    'MAE': [lstm_mae, timesnet_mae],
    'RMSE': [lstm_rmse, timesnet_rmse],
    'R2': [lstm_r2, timesnet_r2]
})
results.to_csv('/content/metrics_summary.csv', index=False)
print("結果儲存至 metrics_summary.csv")

# ========= 10. 預測 vs 實際：散點圖 =========
plt.figure(figsize=(12, 5))

# LSTM
plt.subplot(1, 2, 1)
plt.scatter(ground_truth[:, 0], lstm_pred[:, 0], alpha=0.5, color='blue', label='LSTM 預測')
plt.plot([0, max(ground_truth[:, 0])], [0, max(ground_truth[:, 0])], 'r--', label='理想預測')
plt.xlabel('實際降雨量 (mm)')
plt.ylabel('預測降雨量 (mm)')
plt.title(f'LSTM 預測 vs 實際\nR²={lstm_r2:.3f}')
plt.grid(True)
plt.legend()

# TimesNet
plt.subplot(1, 2, 2)
plt.scatter(ground_truth[:, 0], timesnet_pred[:, 0], alpha=0.5, color='green', label='TimesNet 預測')
plt.plot([0, max(ground_truth[:, 0])], [0, max(ground_truth[:, 0])], 'r--', label='理想預測')
plt.xlabel('實際降雨量 (mm)')
plt.ylabel('預測降雨量 (mm)')
plt.title(f'TimesNet 預測 vs 實際\nR²={timesnet_r2:.3f}')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig('/content/scatter_pred_vs_actual.png')
plt.show()

print("散點圖已儲存為 scatter_pred_vs_actual.png")

